### TorchIP Models 
This notebook shows examples of how to build a (neural network) energy `model` and a `trainer` which consume _energy_ and _force components (gradient)_ to fit the model. 

A neural network performs well when interpolating and NOT extrapolating.

#### TODO:
- [x] neural network + gradient of inputs
- [x] fit example LJ potential
- [x] energy and force data loader
- [ ] GPU version
- [ ] trainer

In [ ]:
import sys
sys.path.append('../')

import math
import torch
from torch import nn
import matplotlib.pylab as plt
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(2022)

# TorchIP imports
import torchip
from torchip.config import CFG # TODO: improve it
from torchip.models import NeuralNetworkModel
from torchip.utils import gradient

In [ ]:
CFG.set("device", "cpu")
print("Device:", CFG["device"])

### Model

In [ ]:
model = NeuralNetworkModel(input_size=3, layers=((8, 't'), (4, 't'), (1, 'l')))
print(model)

In [ ]:
def potential(x, sigma=1.0, epsilon=1.0):
    tmp6 = torch.pow(sigma/torch.norm(x, dim=1), 6)
    return (4.0*epsilon*(tmp6*tmp6 - tmp6)).view(-1, 1)

def get_position(n, requires_grad=True, factor=1.0):
    return torch.rand(n, 3, requires_grad=requires_grad) * factor + 0.5

pos = get_position(1000)   # energy samples
eng = potential(pos)
pos_ = get_position(5000) # force samples
eng_ = potential(pos_)
frc = gradient(eng_, pos_)

In [ ]:
plt.scatter(torch.norm(pos_, dim=1).detach().numpy(), eng_.detach().numpy(), label="gradient")
plt.scatter(torch.norm(pos, dim=1).detach().numpy(), eng.detach().numpy(), label="potential")
plt.xlabel("x"); plt.ylabel("y, dy"); plt.legend(loc='lower right'); plt.title("samples");

In [ ]:
class EnergyDataLoader(Dataset):
    """
    Energy specific DataLoader.
    """
    def __init__(self, position=pos, energy=eng):
        self.position = position
        self.energy = energy

    def __len__(self):
        return len(self.energy)

    def __getitem__(self, idx):
        return {"energy": self.energy[idx, ...], "position": self.position[idx, ...]}
                 
                 
class ForceDataLoader(Dataset):
    """
    Force specific DataLoader.
    """
    def __init__(self, position=pos_, force=frc):
        self.position = position
        self.force = force

    def __len__(self):
        return len(self.force)

    def __getitem__(self, idx):
        return {"force": self.force[idx, ...], "position": self.position[idx, ...]}
    

energy_loader = DataLoader(EnergyDataLoader(), batch_size=100, shuffle=True)
force_loader = DataLoader(ForceDataLoader(), batch_size=500, shuffle=True)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
criterion = nn.MSELoss()

def train(model, optimizer, loss_fn, epochs=1000):
    """
    Train input model using provided optimizer, loss function, and data loader (energy amd forces).
    """
    print(f"Energy update ratio: {len(force_loader)/float(len(energy_loader)):.2f} ({len(force_loader)}/{len(energy_loader)})")
    energy_loader_ = iter(energy_loader)
    
    for epoch in range(epochs):
        strout = f"Epoch: [{epoch+1}/{epochs}]"
        
        # Training 
        training_eng_loss = 0.0
        training_frc_loss = 0.0
        model.train()
        nbatch = 0
        # Loop over force batches
        for index, fbatch in enumerate(force_loader):
            # Reset gradients
            optimizer.zero_grad()
            # Get next energy batch (infinite loader)
            try:
                ebatch = next(energy_loader_)
            except StopIteration:
                # StopIteration is thrown if dataset ends
                # reinitialize data loader 
                #print("Infinite loop")
                energy_loader_ = iter(energy_loader)
                ebatch = next(energy_loader_)
            # Energy    
            eng_ =  model(ebatch["position"])
            eng_loss = criterion(eng_, ebatch["energy"]) 
            # Force
            eng_ = model(fbatch['position'])
            frc_ = gradient(eng_, fbatch['position'])
            frc_loss = criterion(frc_, fbatch['force'])
            # update weights
            loss = eng_loss + frc_loss
            loss.backward(retain_graph=True)
            optimizer.step()
            # Accumulate loss values
            training_eng_loss += eng_loss.data.item()
            training_frc_loss += frc_loss.data.item()
            nbatch += 1
            #print(nbatch, ebatch['energy'].shape, fbatch['force'].shape)
        #assert len(force_loader) > len(energy_loader)
        training_eng_loss /= nbatch
        training_frc_loss /= nbatch
        training_loss = training_eng_loss + training_frc_loss
        strout += f", Training Loss: {training_loss:.8f} <force({training_frc_loss:.8f}), energy({training_eng_loss:.8f})>"
        
        # Validation
        valid_loss = 0.0
        model.eval()
        pos_val = get_position(1000)
        eng_val = potential(pos_val)
        frc_val = gradient(eng_val, pos_val)
        eng_ =  model(pos_val)
        frc_ = gradient(eng_, pos_val)
        loss = criterion(eng_val, eng_) + criterion(frc_val, frc_)
        valid_loss += loss.data.item()
        strout += f", Validation Loss: {valid_loss:.8f}"

        if epoch==0 or (epoch+1) % 100 == 0:
            print(strout)
        

In [ ]:
train(model, optimizer, criterion)

In [ ]:
pos = get_position(5000, factor=1)
plt.scatter(torch.norm(pos, dim=1).detach().numpy(), potential(pos).detach().numpy(), label="potential")
plt.scatter(torch.norm(pos, dim=1).detach().numpy(), model(pos).detach().numpy(), label="neuran network")
plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.title("Energy");

In [ ]:
# pos = get_position(1000)
plt.scatter(torch.norm(pos, dim=1).detach().numpy(), gradient(potential(pos), pos)[:, 0].detach().numpy(), label="potential")
plt.scatter(torch.norm(pos, dim=1).detach().numpy(), gradient(model(pos), pos)[:, 0].detach().numpy(), label="neural network")
plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.title("Force");